In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""

from eos_database import *
from compressor_class import *
from compression import *
from gc_eos_soave import *
from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from viscosity import *
from plenum_system import *

In [2]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)




In [5]:
n_x = 3
n_z = 11

def system_residuals(y, u0, plenum_sys):
    # Separa as variáveis
    x = y[:n_x]
    z = y[n_x:]
    
    # Substitua as expressões abaixo pelas suas equações do modelo em estado estacionário.
    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x, z, u0)
    
    res_ode = np.array([ode_sym[i].item() for i in range(n_x)])
    
    # Calcula os resíduos das equações algébricas
    res_alg = np.array([alg_sym[i] for i in range(n_z)])

    res = np.concatenate((res_ode, res_alg))
    return res

def compute_steady_state(u0, plenum_sys, x0, z0):
    # Vetor inicial concatenado
    y0 = np.array(x0 + z0)
    
    # Chama o fsolve para encontrar os zeros da função de resíduos
    sol = fsolve(system_residuals, y0, args=(u0, plenum_sys))
    
    # Separa a solução em x e z
    x_ss = sol[:n_x]
    z_ss = sol[n_x:]
    return x_ss, z_ss

if __name__ == '__main__':
    Phi, eta, Mach, Gimp, G2, Gdif, PHI, G2s, k = compression.character(compressor, m = 10, N = 750, Gi_1 = gas)
    x0 = [10, G2.T, G2.V]
    print(x0)
    # E z0 pode ser (conforme seu código original):
    z0 = [G2.P, G2.P, Gimp.T, Gimp.V,
          Gdif.T, Gdif.V, G2s.T, G2s.V,
          G2.T, G2.V]+ [gas.V]
    print(z0)
    u0 = [4500, 300, 750, (10/0.38/(G2.P - 5000)**0.5), 5000]
    
    # Calcula os estados estacionários
    x_ss, z_ss = compute_steady_state(u0, plenum_sys, x0, z0)
    print("Estado estacionário (variáveis diferenciais):", x_ss)
    print("Estado estacionário (variáveis algébricas):", z_ss)

[np.float64(-0.00026081045825152537), np.float64(0.012665226711299172), np.float64(-2.9077315736155756e-05), np.float64(1.1083024730194464e-05)]
[10, np.float64(326.52858077220526), np.float64(0.29028879453403433)]
[np.float64(8458.923117067186), np.float64(8458.923117067186), np.float64(314.2023000518625), np.float64(0.3607068213727581), np.float64(313.8219644125313), np.float64(0.36822700873353853), np.float64(324.49499746005193), np.float64(0.2876266933416093), np.float64(326.52858077220526), np.float64(0.29028879453403433), np.float64(0.5098790085836161)]
Estado estacionário (variáveis diferenciais): [1.00000000e+01 3.26528581e+02 2.90288795e-01]
Estado estacionário (variáveis algébricas): [8.45892312e+03 8.45892312e+03 3.14202300e+02 3.60706821e-01
 3.13821964e+02 3.68227009e-01 3.24494997e+02 2.87626693e-01
 3.26528581e+02 2.90288795e-01 5.09879009e-01]


C:\Users\lastb\AppData\Local\Temp\ipykernel_37488\174574149.py:25: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  sol = fsolve(system_residuals, y0, args=(u0, plenum_sys))


In [6]:
def test_evaluate_dae(plenum_sys, x_test, z_test, u0):# Parâmetros de entrada

    # Avaliação da função
    ode_values, alg_values = plenum_sys.evaluate_dae(None, x_test, z_test, u0)

    # Imprimindo os resultados
    print("Valores das ODEs:")
    for i, val in enumerate(ode_values):
        print(f"ODE {i}: {val}")

    print("\nValores das equações algébricas:")
    for i, val in enumerate(alg_values):
        print(f"Algebrica {i}: {val}")

test_evaluate_dae(plenum_sys, x_ss, z_ss, u0)

Valores das ODEs:
ODE 0: 0.0
ODE 1: 0.0
ODE 2: -0.0

Valores das equações algébricas:
Algebrica 0: 0.0
Algebrica 1: 0.0
Algebrica 2: -5.3347191505731385e-11
Algebrica 3: 4.4632849283006097e-11
Algebrica 4: -5.049372111848387e-13
Algebrica 5: 5.059969000365396e-12
Algebrica 6: -0.3475137215623363
Algebrica 7: 2.298755200652238
Algebrica 8: -0.0387437155408179
Algebrica 9: 0.049873611285875086
Algebrica 10: 2.7284841053187847e-12


In [9]:
def run_simulation(x0,z0):
    list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                  "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

    nwe_3 = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0,
             0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

    dict_composition_3 = {list_names[i]: nwe_3[i] for i in range(len(nwe_3))}
    mixture_nwe_3 = Mixture(list_of_species, dict_composition_3)

    volumn_desviation = [0] * 19
    gas = gc_eos_class(mixture_nwe_3, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')  # T inicial em Kelvin

    comp = CompressorClass()
    visc = viscosity(mixture_nwe_3, volumn_desviation)
    compressor = compression(gas, comp, visc)

    vlv = valve(kv=0.38)
    Vpp = 2.0 
    Lc = 2.0 
    A1 = 2.6e-3
    plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)

    dotm_0 = vlv.evaluate_flow((10/0.38/np.sqrt(G2.P - 5000)), 2500)

    # Chutes iniciais (x0: variáveis diferenciais, z0: variáveis algébricas)
    u0 = [4500, 300, 750, (10/0.38/np.sqrt(G2.P - 5000)), 5000]

    # Definição dos símbolos para o DAE
    x_sym = SX.sym('x', 3)
    z_sym = SX.sym('z', 11)
    u_sym = SX.sym('u', 5)

    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x_sym, z_sym, u_sym)

    # total_fun = ca.Function('f',[x_sym,z_sym,u_sym],ode_sym+alg_sym)

    # Fun = lambda y : [f.full().flatten().item() for f in total_fun(y[0:3],y[3:],u0)]
    # print(Fun())
    dae = {
        'x': x_sym,
        'z': z_sym,
        'p': u_sym,
        'ode': vertcat(*ode_sym),
        'alg': vertcat(*alg_sym)
    }

    total_fun = ca.Function('f',[x_sym,z_sym,u_sym],ode_sym+alg_sym)

    print(total_fun(x0,z0,u0))

    t0 = 1
    tf = 100
    # Chamada atualizada do integrador com t0 e tf como argumentos
    integrator_solver = integrator('F', 'idas', dae, 0, 0.1)

    t_sim = 10
    n_steps = int(t_sim / tf)
    time = np.zeros(n_steps + 1)
    dot_m_hist = np.zeros(n_steps + 1)
    Tp_hist = np.zeros(n_steps + 1)
    Pp_hist = np.zeros(n_steps + 1)

    x0_current, z0_current = x0.copy(), z0.copy()
    res = integrator_solver(x0=x0_current, z0=z0_current, p=u0)



    # for i in range(n_steps + 1):
    #     if i > 0:
    #         res = integrator_solver(x0=x0_current, z0=z0_current, p=u0)
    #         x0_current = res['xf'].full().flatten()
    #         z0_current = res['zf'].full().flatten()
    #     time[i] = i * tf
    #     dot_m_hist[i] = x0_current[0]
    #     Tp_hist[i] = x0_current[1]
    #     Pp_hist[i] = z0_current[0]

    # plt.figure(figsize=(10, 8))
    # plt.subplot(3, 1, 1)
    # plt.plot(time, dot_m_hist, 'b-')
    # plt.ylabel('Fluxo de Massa [kg/s]')
    # plt.grid(True)
    
    # plt.subplot(3, 1, 2)
    # plt.plot(time, Tp_hist, 'r-')
    # plt.ylabel('Temperatura [K]')
    # plt.grid(True)
    
    # plt.subplot(3, 1, 3)
    # plt.plot(time, Pp_hist / 1e5, 'g-')
    # plt.ylabel('Pressão [bar]')
    # plt.xlabel('Tempo [s]')
    # plt.grid(True)
    
    # plt.tight_layout()
    # plt.savefig("grafico.png", dpi=300)


run_simulation(x_ss,z_ss)

(DM(0), DM(0), DM(-0), DM(0), DM(0), DM(-5.33472e-11), DM(4.44463e-07), DM(-5.04855e-13), DM(3.71776e-08), DM(-0.347395), DM(2.29876), DM(-0.0388625), DM(0.0498736), DM(2.72848e-12))


CasADi - 2025-03-10 17:36:20 WARNING("F:daeF failed: NaN detected for output ode, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:393]
CasADi - 2025-03-10 17:36:20 WARNING("F:daeF failed: NaN detected for output ode, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:393]
CasADi - 2025-03-10 17:36:20 WARNING("F:daeF failed: NaN detected for output ode, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:393]
CasADi - 2025-03-10 17:36:20 WARNING("F:daeF failed: NaN detected for output ode, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:393]
CasADi - 2025-03-10 17:36:20 WARNING("F:daeF failed: NaN detected for output ode, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:393]
The residual routine or the linear setup or solve routine had a recoverable error, but IDACalcIC was unable to recover.


RuntimeError: Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1432:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:361:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_NO_RECOVERY". Consult IDAS documentation.